1. null, 이상치 제거 (z-score 그래프 보고 3, 4, 5, 6, 7) 다양한 시도 해봐야 함
2. 스케일링 standard, minmax, robust, maxabs + log 스케일링 다 시도해 보아야 함
3. 모델은 여러개 쓰지 않더라도 랜덤, xgboost, lightGBM, gradiant 시도해 보고 -> 스태킹 앙상블 사용해보기
4. 하이퍼 파라미터 조정하기
5. 생각나면 추가적으로

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
import xgboost as xgb
from sklearn.feature_selection import RFECV

In [2]:
df = pd.read_csv('../game_result/merged_FBREF_train_validation_set.csv')

In [3]:
df

,Date,Team,Opponent,Referee,TY,OY,TR,OR,B365TW,B365D,...,Clean Sheet,Pass Completion %,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error
0,2018-08-10,Leicester City,Manchester Utd,A Marriner,1,2,0,0,7.50,3.90,...,0,79.4,0,0.8,1.0,11,21.4,8,12,0
1,2018-08-10,Manchester Utd,Leicester City,A Marriner,2,1,0,0,1.57,3.90,...,0,80.1,1,0.7,0.3,13,50.0,15,17,0
2,2018-08-11,Bournemouth,Cardiff City,K Friend,1,1,0,0,1.90,3.60,...,1,79.0,2,1.3,1.1,5,47.1,11,4,0
3,2018-08-11,Brighton & Hove Albion,Watford,J Moss,2,2,0,0,3.40,3.20,...,0,74.4,0,0.3,0.2,11,50.0,14,17,0
4,2018-08-11,Cardiff City,Bournemouth,K Friend,1,1,0,0,4.50,3.60,...,0,58.4,0,1.5,1.2,12,47.6,16,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4465,2024-04-24,Everton,Liverpool,A Madley,0,3,0,0,7.00,5.00,...,1,63.1,1,0.8,0.6,9,42.9,14,14,0
4466,2024-04-24,Sheffield Utd,Manchester Utd,M Salisbury,1,0,0,0,8.50,6.00,...,0,72.2,1,0.5,0.3,9,46.7,10,13,0
4467,2024-04-24,Crystal Palace,Newcastle Utd,T Bramall,3,3,0,0,3.10,3.60,...,1,74.5,2,0.7,0.8,19,60.0,15,6,0
4468,2024-04-24,Manchester Utd,Sheffield Utd,M Salisbury,0,1,0,0,1.30,6.00,...,0,87.6,3,2.0,1.8,9,52.9,6,5,1


In [4]:
df.columns

Index(['Date', 'Team', 'Opponent', 'Referee', 'TY', 'OY', 'TR', 'OR', 'B365TW',
       'B365D', 'B365OW', 'Season', 'Time', 'Round', 'Round.1', 'Table', 'Day',
       'Venue', 'Result', 'Possesion', 'Aerial Duels(%)', 'GF', 'GA',
       'Oppo. Table', 'Shot on Target', 'Shot on Target(%)', 'Goals per Shot',
       'Expected Goals', 'Save%', 'Clean Sheet', 'Pass Completion %',
       'Assists', 'Exp. Assisted Goals', 'Expected Assists', 'Tackles Won',
       '% of Dribblers Tackled', 'Blocks', 'Interceptions', 'Error'],
      dtype='object')

In [5]:
df.isnull().sum()

Date                      0
Team                      0
Opponent                  0
Referee                   0
TY                        0
OY                        0
TR                        0
OR                        0
B365TW                    0
B365D                     0
B365OW                    0
Season                    0
Time                      0
Round                     0
Round.1                   0
Table                     0
Day                       0
Venue                     0
Result                    0
Possesion                 0
Aerial Duels(%)           0
GF                        0
GA                        0
Oppo. Table               0
Shot on Target            0
Shot on Target(%)         2
Goals per Shot            2
Expected Goals            0
Save%                     0
Clean Sheet               0
Pass Completion %         0
Assists                   0
Exp. Assisted Goals       0
Expected Assists          0
Tackles Won               0
% of Dribblers Tackl

In [6]:
df = df.fillna(0)

In [7]:
df.isnull().sum()

Date                      0
Team                      0
Opponent                  0
Referee                   0
TY                        0
OY                        0
TR                        0
OR                        0
B365TW                    0
B365D                     0
B365OW                    0
Season                    0
Time                      0
Round                     0
Round.1                   0
Table                     0
Day                       0
Venue                     0
Result                    0
Possesion                 0
Aerial Duels(%)           0
GF                        0
GA                        0
Oppo. Table               0
Shot on Target            0
Shot on Target(%)         0
Goals per Shot            0
Expected Goals            0
Save%                     0
Clean Sheet               0
Pass Completion %         0
Assists                   0
Exp. Assisted Goals       0
Expected Assists          0
Tackles Won               0
% of Dribblers Tackl

In [8]:
# 지금 이 데이터셋에는 감독이 없는데 club_games 데이터셋의 감독을 넣어서 모델링
# -> club_games, games dataset 가져와야 함

In [9]:
club_games_df = pd.read_csv('club_games.csv')

In [10]:
club_games_df

,game_id,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
0,2320450,1468,0,NaN,Holger Bachthaler,24,2,NaN,Armin Veh,Home,0
1,2320460,1,3,NaN,Jürgen Luginger,86,1,NaN,Robin Dutt,Home,1
2,2320472,2036,4,NaN,Frank Schmidt,72,5,NaN,Alexander Schmidt,Home,0
3,2321044,16,2,1.0,Jürgen Klopp,23,1,15.0,Torsten Lieberknecht,Home,1
4,2321060,23,0,18.0,Torsten Lieberknecht,24,2,11.0,Armin Veh,Home,0
...,...,...,...,...,...,...,...,...,...,...,...
135511,3142950,681,0,NaN,Imanol Alguacil,150,0,NaN,Quique Setién,Away,0
135512,3148382,150,1,NaN,Quique Setién,714,1,NaN,Rubi,Away,0
135513,3153158,418,1,NaN,Santiago Solari,131,1,NaN,Ernesto Valverde,Away,0
135514,3307741,36661,1,NaN,Aleksandr Gorshkov,32218,2,NaN,Viktor Antikhovich,Away,0


In [11]:
club_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135516 entries, 0 to 135515
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   game_id                135516 non-null  int64  
 1   club_id                135516 non-null  int64  
 2   own_goals              135516 non-null  int64  
 3   own_position           95480 non-null   float64
 4   own_manager_name       134026 non-null  object 
 5   opponent_id            135516 non-null  int64  
 6   opponent_goals         135516 non-null  int64  
 7   opponent_position      95480 non-null   float64
 8   opponent_manager_name  134026 non-null  object 
 9   hosting                135516 non-null  object 
 10  is_win                 135516 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 11.4+ MB


In [12]:
games_df = pd.read_csv('games.csv')

In [13]:
games_df

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
0,2321044,L1,2013,2. Matchday,2013-08-18,16,23,2,1,1.0,...,SIGNAL IDUNA PARK,80200.0,Peter Sippel,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,4-3-2-1,Borussia Dortmund,Eintracht Braunschweig,2:1,domestic_league
1,2321060,L1,2013,3. Matchday,2013-08-25,23,24,0,2,18.0,...,EINTRACHT-Stadion,23325.0,Wolfgang Stark,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-2-3-1,Eintracht Braunschweig,Eintracht Frankfurt Fußball AG,0:2,domestic_league
2,2321086,L1,2013,6. Matchday,2013-09-21,4,16,1,1,15.0,...,Max-Morlock-Stadion,50000.0,Knut Kircher,https://www.transfermarkt.co.uk/1-fc-nuremberg...,4-2-3-1,4-2-3-1,1.FC Nuremberg,Borussia Dortmund,1:1,domestic_league
3,2321152,L1,2013,11. Matchday,2013-11-02,44,33,0,2,7.0,...,Olympiastadion Berlin,69277.0,Günter Perl,https://www.transfermarkt.co.uk/hertha-bsc_fc-...,4-2-3-1,4-2-3-1,Hertha BSC,FC Schalke 04,0:2,domestic_league
4,2321205,L1,2013,17. Matchday,2013-12-21,41,39,2,3,14.0,...,Volksparkstadion,50000.0,Bastian Dankert,https://www.transfermarkt.co.uk/hamburger-sv_1...,4-2-3-1,4-4-2 Diamond,Hamburger SV,1. Fußball- und Sportverein Mainz 05,2:3,domestic_league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67753,2700827,NL1,2016,23. Matchday,2017-02-18,383,467,3,1,3.0,...,Philips Stadion,33500.0,Jochem Kamphuis,https://www.transfermarkt.co.uk/psv-eindhoven_...,4-3-3 Attacking,5-3-2,Eindhovense Voetbalvereniging Philips Sport Ve...,Nijmegen Eendracht Combinatie,3:1,domestic_league
67754,3852741,NL1,2022,27. Matchday,2023-04-01,467,383,2,4,10.0,...,Goffertstadion,12500.0,Edwin van de Graaf,https://www.transfermarkt.co.uk/nec-nijmegen_p...,4-3-3 Defending,4-3-3 Defending,Nijmegen Eendracht Combinatie,Eindhovense Voetbalvereniging Philips Sport Ve...,2:4,domestic_league
67755,4097959,NL1,2023,5. Matchday,2023-09-16,383,467,4,0,1.0,...,Philips Stadion,34200.0,Joey Kooij,https://www.transfermarkt.co.uk/psv-eindhoven_...,4-3-3 Attacking,4-3-3 Defending,Eindhovense Voetbalvereniging Philips Sport Ve...,Nijmegen Eendracht Combinatie,4:0,domestic_league
67756,4098179,NL1,2023,27. Matchday,2024-03-30,467,383,3,1,6.0,...,Goffertstadion,NaN,Bas Nijhuis,https://www.transfermarkt.co.uk/nec-nijmegen_p...,4-3-3 Defending,4-3-3 Defending,Nijmegen Eendracht Combinatie,Eindhovense Voetbalvereniging Philips Sport Ve...,3:1,domestic_league


In [14]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67758 entries, 0 to 67757
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_id                 67758 non-null  int64  
 1   competition_id          67758 non-null  object 
 2   season                  67758 non-null  int64  
 3   round                   67758 non-null  object 
 4   date                    67758 non-null  object 
 5   home_club_id            67758 non-null  int64  
 6   away_club_id            67758 non-null  int64  
 7   home_club_goals         67758 non-null  int64  
 8   away_club_goals         67758 non-null  int64  
 9   home_club_position      47740 non-null  float64
 10  away_club_position      47740 non-null  float64
 11  home_club_manager_name  67013 non-null  object 
 12  away_club_manager_name  67013 non-null  object 
 13  stadium                 67544 non-null  object 
 14  attendance              58083 non-null

In [15]:
games_df = games_df[games_df['competition_id'] == 'GB1']

In [16]:
games_df

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
24,2332021,GB1,2013,17. Matchday,2013-12-21,984,3008,1,1,16.0,...,The Hawthorns,24753.0,Jonathan Moss,https://www.transfermarkt.co.uk/west-bromwich-...,4-2-3-1,5-3-2,West Bromwich Albion,Hull City,1:1,domestic_league
25,2332050,GB1,2013,14. Matchday,2013-12-04,289,631,3,4,20.0,...,Stadium of Light,40652.0,Phil Dowd,https://www.transfermarkt.co.uk/sunderland-afc...,4-1-4-1,4-2-3-1,Sunderland AFC,Chelsea Football Club,3:4,domestic_league
26,2332053,GB1,2013,13. Matchday,2013-11-30,379,931,3,0,15.0,...,Boleyn Ground,34946.0,Martin Atkinson,https://www.transfermarkt.co.uk/west-ham-unite...,4-2-3-1,4-4-1-1,West Ham United Football Club,Fulham Football Club,3:0,domestic_league
27,2332188,GB1,2013,23. Matchday,2014-01-28,180,11,2,2,9.0,...,St Mary's Stadium,31284.0,Lee Mason,https://www.transfermarkt.co.uk/southampton-fc...,4-2-3-1,4-2-3-1,Southampton FC,Arsenal Football Club,2:2,domestic_league
28,2332235,GB1,2013,23. Matchday,2014-01-28,873,3008,1,0,14.0,...,Selhurst Park,22519.0,Roger East,https://www.transfermarkt.co.uk/crystal-palace...,4-4-2 double 6,3-5-2 flat,Crystal Palace Football Club,Hull City,1:0,domestic_league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67518,2872227,GB1,2017,12. Matchday,2017-11-20,1237,512,2,2,9.0,...,AMEX Stadium,29676.0,Lee Mason,https://www.transfermarkt.co.uk/brighton-amp-h...,4-4-2 double 6,3-5-2,Brighton and Hove Albion Football Club,Stoke City,2:2,domestic_league
67519,2872273,GB1,2017,16. Matchday,2017-12-09,148,512,5,1,6.0,...,Wembley Stadium,62202.0,Roger East,https://www.transfermarkt.co.uk/tottenham-hots...,4-2-3-1,5-4-1,Tottenham Hotspur Football Club,Stoke City,5:1,domestic_league
67644,4095403,GB1,2023,33. Matchday,2024-04-13,1132,1237,1,1,19.0,...,Turf Moor,NaN,Simon Hooper,https://www.transfermarkt.co.uk/burnley-fc_bri...,4-4-2,4-2-3-1,Burnley Football Club,Brighton and Hove Albion Football Club,1:1,domestic_league
67649,2872297,GB1,2017,19. Matchday,2017-12-23,1132,148,0,3,7.0,...,Turf Moor,21650.0,Michael Oliver,https://www.transfermarkt.co.uk/burnley-fc_tot...,4-2-3-1,4-2-3-1,Burnley Football Club,Tottenham Hotspur Football Club,0:3,domestic_league


In [17]:
games_df['date'] = pd.to_datetime(games_df['date'])

In [18]:
games_df = games_df[games_df['date'] >= '2018-08-01']

In [19]:
games_df

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
1876,3050170,GB1,2018,1. Matchday,2018-08-11,1110,631,0,3,19.0,...,John Smith's Stadium,24121.0,Chris Kavanagh,https://www.transfermarkt.co.uk/huddersfield-t...,3-5-2 flat,4-3-3 Attacking,Huddersfield Town,Chelsea Football Club,0:3,domestic_league
1877,3050215,GB1,2018,5. Matchday,2018-09-15,1010,985,1,2,4.0,...,Vicarage Road,20537.0,Mike Dean,https://www.transfermarkt.co.uk/watford-fc_man...,4-4-2,4-2-3-1,Watford FC,Manchester United Football Club,1:2,domestic_league
1878,3050250,GB1,2018,9. Matchday,2018-10-20,631,985,2,2,3.0,...,Stamford Bridge,40721.0,Mike Dean,https://www.transfermarkt.co.uk/chelsea-fc_man...,4-3-3 Attacking,4-3-3 Attacking,Chelsea Football Club,Manchester United Football Club,2:2,domestic_league
1879,3050264,GB1,2018,10. Matchday,2018-10-27,180,762,0,0,16.0,...,St Mary's Stadium,30736.0,Chris Kavanagh,https://www.transfermarkt.co.uk/southampton-fc...,4-4-2 double 6,4-4-2 double 6,Southampton FC,Newcastle United Football Club,0:0,domestic_league
1880,3050336,GB1,2018,17. Matchday,2018-12-15,543,989,2,0,7.0,...,Molineux Stadium,30997.0,Simon Hooper,https://www.transfermarkt.co.uk/wolverhampton-...,3-5-2,3-4-2-1,Wolverhampton Wanderers Football Club,Association Football Club Bournemouth,2:0,domestic_league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67257,3429714,GB1,2020,23. Matchday,2021-02-06,1132,1237,1,1,17.0,...,Turf Moor,NaN,Anthony Taylor,https://www.transfermarkt.co.uk/burnley-fc_bri...,4-4-2 double 6,5-3-2,Burnley Football Club,Brighton and Hove Albion Football Club,1:1,domestic_league
67504,3429694,GB1,2020,21. Matchday,2021-01-31,1237,148,1,0,17.0,...,AMEX Stadium,NaN,Peter Bankes,https://www.transfermarkt.co.uk/brighton-amp-h...,3-4-2-1,3-4-2-1,Brighton and Hove Albion Football Club,Tottenham Hotspur Football Club,1:0,domestic_league
67512,3592240,GB1,2021,37. Matchday,2022-05-15,148,1132,1,0,4.0,...,Tottenham Hotspur Stadium,61729.0,Kevin Friend,https://www.transfermarkt.co.uk/tottenham-hots...,5-4-1,5-3-2,Tottenham Hotspur Football Club,Burnley Football Club,1:0,domestic_league
67644,4095403,GB1,2023,33. Matchday,2024-04-13,1132,1237,1,1,19.0,...,Turf Moor,NaN,Simon Hooper,https://www.transfermarkt.co.uk/burnley-fc_bri...,4-4-2,4-2-3-1,Burnley Football Club,Brighton and Hove Albion Football Club,1:1,domestic_league


In [20]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2223 entries, 1876 to 67718
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   game_id                 2223 non-null   int64         
 1   competition_id          2223 non-null   object        
 2   season                  2223 non-null   int64         
 3   round                   2223 non-null   object        
 4   date                    2223 non-null   datetime64[ns]
 5   home_club_id            2223 non-null   int64         
 6   away_club_id            2223 non-null   int64         
 7   home_club_goals         2223 non-null   int64         
 8   away_club_goals         2223 non-null   int64         
 9   home_club_position      2223 non-null   float64       
 10  away_club_position      2223 non-null   float64       
 11  home_club_manager_name  2223 non-null   object        
 12  away_club_manager_name  2223 non-null   object   

In [21]:
games_df.isnull().sum()

game_id                     0
competition_id              0
season                      0
round                       0
date                        0
home_club_id                0
away_club_id                0
home_club_goals             0
away_club_goals             0
home_club_position          0
away_club_position          0
home_club_manager_name      0
away_club_manager_name      0
stadium                     0
attendance                464
referee                     0
url                         0
home_club_formation         0
away_club_formation         0
home_club_name              0
away_club_name              0
aggregate                   0
competition_type            0
dtype: int64

In [22]:
club_games_df = club_games_df[club_games_df['game_id'].isin(games_df['game_id'])]

In [23]:
club_games_df

,game_id,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
2096,3050170,1110,0,19.0,David Wagner,631,3,2.0,Maurizio Sarri,Home,0
2097,3050190,1110,0,19.0,David Wagner,603,0,15.0,Neil Warnock,Home,0
2098,3050199,603,2,16.0,Neil Warnock,11,3,9.0,Unai Emery,Home,0
2099,3050215,1010,1,4.0,Javi Gracia,985,2,8.0,José Mourinho,Home,0
2100,3050250,631,2,3.0,Maurizio Sarri,985,2,10.0,José Mourinho,Home,0
...,...,...,...,...,...,...,...,...,...,...,...
135489,3429732,350,0,20.0,Chris Wilder,379,3,5.0,David Moyes,Away,0
135490,3429776,399,2,11.0,Marcelo Bielsa,931,1,18.0,Scott Parker,Away,1
135492,3592178,31,2,2.0,Jürgen Klopp,405,1,15.0,Steven Gerrard,Away,1
135495,4095319,1148,2,14.0,Thomas Frank,543,0,11.0,Gary O'Neil,Away,1


In [24]:
# club_games_df에 date가 필요해서 merge함

merged_df1 = pd.merge(games_df[['game_id', 'date']], club_games_df, on='game_id', how='right')

In [25]:
merged_df1.sort_values('game_id')

,game_id,date,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
933,3050167,2018-08-12,11,0,15.0,Unai Emery,281,2,3.0,Pep Guardiola,Home,0
3156,3050167,2018-08-12,281,2,3.0,Pep Guardiola,11,0,15.0,Unai Emery,Away,1
4265,3050168,2018-08-11,603,0,17.0,Neil Warnock,989,2,5.0,Eddie Howe,Away,0
2042,3050168,2018-08-11,989,2,5.0,Eddie Howe,603,0,17.0,Neil Warnock,Home,1
934,3050169,2018-08-11,931,0,18.0,Slavisa Jokanovic,873,2,4.0,Roy Hodgson,Home,0
...,...,...,...,...,...,...,...,...,...,...,...,...
912,4095407,2024-04-13,762,4,6.0,Eddie Howe,148,0,5.0,Ange Postecoglou,Home,1
2379,4095408,2024-04-13,543,2,11.0,Gary O'Neil,703,2,17.0,Nuno Espírito Santo,Away,0
156,4095408,2024-04-13,703,2,17.0,Nuno Espírito Santo,543,2,11.0,Gary O'Neil,Home,0
4426,4095409,2024-04-14,931,2,12.0,Marco Silva,379,0,8.0,David Moyes,Away,1


In [26]:
merged_home_df = merged_df1[merged_df1['hosting'] == 'Home']

In [27]:
merged_away_df = merged_df1[merged_df1['hosting'] == 'Away']

In [28]:
home_game_df = pd.merge(merged_home_df, games_df[['game_id', 'home_club_name']], on='game_id')

In [29]:
home_game_df = home_game_df.sort_values('game_id')

In [30]:
new_column_names = {'home_club_name': 'club_name'}
home_game_df = home_game_df.rename(columns=new_column_names)

In [31]:
away_game_df = pd.merge(merged_away_df, games_df[['game_id', 'away_club_name']], on='game_id')

In [32]:
away_game_df = away_game_df.sort_values('game_id')

In [33]:
new_column_names = {'away_club_name': 'club_name'}
away_game_df = away_game_df.rename(columns=new_column_names)

In [36]:
m_merged_df = pd.concat([home_game_df, away_game_df], axis=0).sort_values('game_id')

In [37]:
# 이 df가 games, club_games 합친 df
m_merged_df

,game_id,date,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win,club_name
933,3050167,2018-08-12,11,0,15.0,Unai Emery,281,2,3.0,Pep Guardiola,Home,0,Arsenal Football Club
933,3050167,2018-08-12,281,2,3.0,Pep Guardiola,11,0,15.0,Unai Emery,Away,1,Manchester City Football Club
2042,3050168,2018-08-11,603,0,17.0,Neil Warnock,989,2,5.0,Eddie Howe,Away,0,Cardiff City
2042,3050168,2018-08-11,989,2,5.0,Eddie Howe,603,0,17.0,Neil Warnock,Home,1,Association Football Club Bournemouth
934,3050169,2018-08-11,931,0,18.0,Slavisa Jokanovic,873,2,4.0,Roy Hodgson,Home,0,Fulham Football Club
...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,4095407,2024-04-13,762,4,6.0,Eddie Howe,148,0,5.0,Ange Postecoglou,Home,1,Newcastle United Football Club
156,4095408,2024-04-13,543,2,11.0,Gary O'Neil,703,2,17.0,Nuno Espírito Santo,Away,0,Wolverhampton Wanderers Football Club
156,4095408,2024-04-13,703,2,17.0,Nuno Espírito Santo,543,2,11.0,Gary O'Neil,Home,0,Nottingham Forest Football Club
2203,4095409,2024-04-14,379,0,8.0,David Moyes,931,2,12.0,Marco Silva,Home,0,West Ham United Football Club


In [38]:
dropped_df = df.drop(['Season', 'Time', 'Round', 'Round.1', 'Day'], axis=1)

In [39]:
dropped_df.columns

Index(['Date', 'Team', 'Opponent', 'Referee', 'TY', 'OY', 'TR', 'OR', 'B365TW',
       'B365D', 'B365OW', 'Table', 'Venue', 'Result', 'Possesion',
       'Aerial Duels(%)', 'GF', 'GA', 'Oppo. Table', 'Shot on Target',
       'Shot on Target(%)', 'Goals per Shot', 'Expected Goals', 'Save%',
       'Clean Sheet', 'Pass Completion %', 'Assists', 'Exp. Assisted Goals',
       'Expected Assists', 'Tackles Won', '% of Dribblers Tackled', 'Blocks',
       'Interceptions', 'Error'],
      dtype='object')

In [40]:
# df 와 merge하기 위해서 club이름을 다시 설정

In [41]:
name_mapping = {
    'Arsenal Football Club': 'Arsenal', 
    'Manchester City Football Club': 'Manchester City',
    'Cardiff City': 'Cardiff City', 
    'Association Football Club Bournemouth': 'Bournemouth',
    'Fulham Football Club': 'Fulham', 
    'Crystal Palace Football Club': 'Crystal Palace',
    'Huddersfield Town': 'Huddersfield', 
    'Chelsea Football Club': 'Chelsea',
    'West Ham United Football Club': 'West Ham', 
    'Liverpool Football Club': 'Liverpool',
    'Manchester United Football Club': 'Manchester Utd', 
    'Leicester City': 'Leicester City',
    'Newcastle United Football Club': 'Newcastle Utd',
    'Tottenham Hotspur Football Club': 'Tottenham', 
    'Burnley Football Club': 'Burnley',
    'Southampton FC': 'Southampton', 
    'Watford FC': 'Watford',
    'Brighton and Hove Albion Football Club': 'Brighton & Hove Albion',
    'Wolverhampton Wanderers Football Club': 'Wolverhampton', 
    'Everton Football Club': 'Everton',
    'Norwich City': 'Norwich City', 
    'Sheffield United Football Club': 'Sheffield Utd',
    'Aston Villa Football Club': 'Aston Villa', 
    'Leeds United': 'Leeds United',
    'West Bromwich Albion': 'West Bromwich Albion', 
    'Brentford Football Club': 'Brentford',
    'Nottingham Forest Football Club': 'Nottingham Forest', 
    'Luton Town Football Club': 'Luton Town'
}

In [42]:
m_merged_df['club_name'] = m_merged_df['club_name'].replace(name_mapping)

In [43]:
m_merged_df

,game_id,date,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win,club_name
933,3050167,2018-08-12,11,0,15.0,Unai Emery,281,2,3.0,Pep Guardiola,Home,0,Arsenal
933,3050167,2018-08-12,281,2,3.0,Pep Guardiola,11,0,15.0,Unai Emery,Away,1,Manchester City
2042,3050168,2018-08-11,603,0,17.0,Neil Warnock,989,2,5.0,Eddie Howe,Away,0,Cardiff City
2042,3050168,2018-08-11,989,2,5.0,Eddie Howe,603,0,17.0,Neil Warnock,Home,1,Bournemouth
934,3050169,2018-08-11,931,0,18.0,Slavisa Jokanovic,873,2,4.0,Roy Hodgson,Home,0,Fulham
...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,4095407,2024-04-13,762,4,6.0,Eddie Howe,148,0,5.0,Ange Postecoglou,Home,1,Newcastle Utd
156,4095408,2024-04-13,543,2,11.0,Gary O'Neil,703,2,17.0,Nuno Espírito Santo,Away,0,Wolverhampton
156,4095408,2024-04-13,703,2,17.0,Nuno Espírito Santo,543,2,11.0,Gary O'Neil,Home,0,Nottingham Forest
2203,4095409,2024-04-14,379,0,8.0,David Moyes,931,2,12.0,Marco Silva,Home,0,West Ham


In [44]:
# 내가 지금까지 club_games랑 games df를 전처리한 건 df DataFrame에 감독이름을 추가하기 위해서

In [45]:
# 다음은 df dataframe 정리하기

In [46]:
# 배팅 정보는 확률로 변경

In [47]:
dropped_df['B365TW'] = (1/dropped_df['B365TW'])*100

In [48]:
dropped_df['B365D'] = (1/dropped_df['B365D'])*100

In [49]:
dropped_df['B365OW'] = (1/dropped_df['B365OW'])*100

In [50]:
dropped_df.columns

Index(['Date', 'Team', 'Opponent', 'Referee', 'TY', 'OY', 'TR', 'OR', 'B365TW',
       'B365D', 'B365OW', 'Table', 'Venue', 'Result', 'Possesion',
       'Aerial Duels(%)', 'GF', 'GA', 'Oppo. Table', 'Shot on Target',
       'Shot on Target(%)', 'Goals per Shot', 'Expected Goals', 'Save%',
       'Clean Sheet', 'Pass Completion %', 'Assists', 'Exp. Assisted Goals',
       'Expected Assists', 'Tackles Won', '% of Dribblers Tackled', 'Blocks',
       'Interceptions', 'Error'],
      dtype='object')

In [51]:
dropped_df.head()

,Date,Team,Opponent,Referee,TY,OY,TR,OR,B365TW,B365D,...,Clean Sheet,Pass Completion %,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error
0,2018-08-10,Leicester City,Manchester Utd,A Marriner,1,2,0,0,13.333333,25.641026,...,0,79.4,0,0.8,1.0,11,21.4,8,12,0
1,2018-08-10,Manchester Utd,Leicester City,A Marriner,2,1,0,0,63.694268,25.641026,...,0,80.1,1,0.7,0.3,13,50.0,15,17,0
2,2018-08-11,Bournemouth,Cardiff City,K Friend,1,1,0,0,52.631579,27.777778,...,1,79.0,2,1.3,1.1,5,47.1,11,4,0
3,2018-08-11,Brighton & Hove Albion,Watford,J Moss,2,2,0,0,29.411765,31.250000,...,0,74.4,0,0.3,0.2,11,50.0,14,17,0
4,2018-08-11,Cardiff City,Bournemouth,K Friend,1,1,0,0,22.222222,27.777778,...,0,58.4,0,1.5,1.2,12,47.6,16,15,0


In [52]:
dropped_df['Date'] = pd.to_datetime(dropped_df['Date'])

In [53]:
master_df = pd.merge(m_merged_df[['game_id', 'date', 'club_name', 'club_id', 'opponent_id', 'own_manager_name', 'opponent_manager_name']], dropped_df, \
         left_on=['date', 'club_name'], right_on=['Date', 'Team'])

In [54]:
master_df = master_df.drop(['date', 'club_name'], axis=1)

In [55]:
master_df.columns

Index(['game_id', 'club_id', 'opponent_id', 'own_manager_name',
       'opponent_manager_name', 'Date', 'Team', 'Opponent', 'Referee', 'TY',
       'OY', 'TR', 'OR', 'B365TW', 'B365D', 'B365OW', 'Table', 'Venue',
       'Result', 'Possesion', 'Aerial Duels(%)', 'GF', 'GA', 'Oppo. Table',
       'Shot on Target', 'Shot on Target(%)', 'Goals per Shot',
       'Expected Goals', 'Save%', 'Clean Sheet', 'Pass Completion %',
       'Assists', 'Exp. Assisted Goals', 'Expected Assists', 'Tackles Won',
       '% of Dribblers Tackled', 'Blocks', 'Interceptions', 'Error'],
      dtype='object')

In [56]:
new_column_order = ['game_id', 'Date', 'Team', 'Opponent', 'club_id', 'opponent_id', 'own_manager_name', 'opponent_manager_name',
       'Referee', 'TY', 'OY', 'TR', 'OR', 'B365TW', 'B365D',
       'B365OW', 'Table', 'Venue', 'Result', 'Possesion', 'Aerial Duels(%)',
       'GF', 'GA', 'Oppo. Table', 'Shot on Target', 'Shot on Target(%)',
       'Goals per Shot', 'Expected Goals', 'Save%', 'Clean Sheet',
       'Pass Completion %', 'Assists', 'Exp. Assisted Goals',
       'Expected Assists', 'Tackles Won', '% of Dribblers Tackled', 'Blocks',
       'Interceptions', 'Error']

In [57]:
master_df

,game_id,club_id,opponent_id,own_manager_name,opponent_manager_name,Date,Team,Opponent,Referee,TY,...,Clean Sheet,Pass Completion %,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error
0,3050167,11,281,Unai Emery,Pep Guardiola,2018-08-12,Arsenal,Manchester City,M Oliver,2,...,0,77.1,0,0.5,0.6,14,40.0,13,12,1
1,3050167,281,11,Pep Guardiola,Unai Emery,2018-08-12,Manchester City,Arsenal,M Oliver,2,...,1,81.9,2,0.5,0.4,5,28.6,14,12,1
2,3050168,603,989,Neil Warnock,Eddie Howe,2018-08-11,Cardiff City,Bournemouth,K Friend,1,...,0,58.4,0,1.5,1.2,12,47.6,16,15,0
3,3050168,989,603,Eddie Howe,Neil Warnock,2018-08-11,Bournemouth,Cardiff City,K Friend,1,...,1,79.0,2,1.3,1.1,5,47.1,11,4,0
4,3050169,931,873,Slavisa Jokanovic,Roy Hodgson,2018-08-11,Fulham,Crystal Palace,M Dean,1,...,0,85.2,0,0.5,0.7,19,42.9,5,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4427,4095402,1148,350,Thomas Frank,Chris Wilder,2024-04-13,Brentford,Sheffield Utd,S Barrott,3,...,1,74.4,1,1.0,1.1,8,21.7,15,14,0
4428,4095403,1237,1132,Roberto De Zerbi,Vincent Kompany,2024-04-13,Brighton & Hove Albion,Burnley,S Hooper,1,...,0,84.2,0,0.5,0.9,8,64.7,11,5,1
4429,4095403,1132,1237,Vincent Kompany,Roberto De Zerbi,2024-04-13,Burnley,Brighton & Hove Albion,S Hooper,1,...,0,79.7,0,1.5,1.5,6,61.9,15,5,1
4430,4095406,1031,281,Rob Edwards,Pep Guardiola,2024-04-13,Luton Town,Manchester City,J Brooks,1,...,0,76.0,0,0.3,0.1,7,44.4,18,8,0


In [58]:
master_df = master_df[new_column_order]

In [59]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4432 entries, 0 to 4431
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   game_id                 4432 non-null   int64         
 1   Date                    4432 non-null   datetime64[ns]
 2   Team                    4432 non-null   object        
 3   Opponent                4432 non-null   object        
 4   club_id                 4432 non-null   int64         
 5   opponent_id             4432 non-null   int64         
 6   own_manager_name        4432 non-null   object        
 7   opponent_manager_name   4432 non-null   object        
 8   Referee                 4432 non-null   object        
 9   TY                      4432 non-null   int64         
 10  OY                      4432 non-null   int64         
 11  TR                      4432 non-null   int64         
 12  OR                      4432 non-null   int64   

In [ ]:
--------------

In [60]:
# master_df 에 더해서 

In [60]:
club_value_by_game = pd.read_csv('mean_value_by_game(train+validation).csv')

In [61]:
master_df.head()

,game_id,Date,Team,Opponent,club_id,opponent_id,own_manager_name,opponent_manager_name,Referee,TY,...,Clean Sheet,Pass Completion %,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error
0,3050167,2018-08-12,Arsenal,Manchester City,11,281,Unai Emery,Pep Guardiola,M Oliver,2,...,0,77.1,0,0.5,0.6,14,40.0,13,12,1
1,3050167,2018-08-12,Manchester City,Arsenal,281,11,Pep Guardiola,Unai Emery,M Oliver,2,...,1,81.9,2,0.5,0.4,5,28.6,14,12,1
2,3050168,2018-08-11,Cardiff City,Bournemouth,603,989,Neil Warnock,Eddie Howe,K Friend,1,...,0,58.4,0,1.5,1.2,12,47.6,16,15,0
3,3050168,2018-08-11,Bournemouth,Cardiff City,989,603,Eddie Howe,Neil Warnock,K Friend,1,...,1,79.0,2,1.3,1.1,5,47.1,11,4,0
4,3050169,2018-08-11,Fulham,Crystal Palace,931,873,Slavisa Jokanovic,Roy Hodgson,M Dean,1,...,0,85.2,0,0.5,0.7,19,42.9,5,15,1


In [62]:
club_value_by_game.head()

,game_id,player_club_id,market_value_in_eur
0,3050167,11,3.064286e+07
1,3050167,281,6.357143e+07
2,3050168,603,2.196429e+06
3,3050168,989,6.035714e+06
4,3050169,873,1.117857e+07


In [63]:
master_df

,game_id,Date,Team,Opponent,club_id,opponent_id,own_manager_name,opponent_manager_name,Referee,TY,...,Clean Sheet,Pass Completion %,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error
0,3050167,2018-08-12,Arsenal,Manchester City,11,281,Unai Emery,Pep Guardiola,M Oliver,2,...,0,77.1,0,0.5,0.6,14,40.0,13,12,1
1,3050167,2018-08-12,Manchester City,Arsenal,281,11,Pep Guardiola,Unai Emery,M Oliver,2,...,1,81.9,2,0.5,0.4,5,28.6,14,12,1
2,3050168,2018-08-11,Cardiff City,Bournemouth,603,989,Neil Warnock,Eddie Howe,K Friend,1,...,0,58.4,0,1.5,1.2,12,47.6,16,15,0
3,3050168,2018-08-11,Bournemouth,Cardiff City,989,603,Eddie Howe,Neil Warnock,K Friend,1,...,1,79.0,2,1.3,1.1,5,47.1,11,4,0
4,3050169,2018-08-11,Fulham,Crystal Palace,931,873,Slavisa Jokanovic,Roy Hodgson,M Dean,1,...,0,85.2,0,0.5,0.7,19,42.9,5,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4427,4095402,2024-04-13,Brentford,Sheffield Utd,1148,350,Thomas Frank,Chris Wilder,S Barrott,3,...,1,74.4,1,1.0,1.1,8,21.7,15,14,0
4428,4095403,2024-04-13,Brighton & Hove Albion,Burnley,1237,1132,Roberto De Zerbi,Vincent Kompany,S Hooper,1,...,0,84.2,0,0.5,0.9,8,64.7,11,5,1
4429,4095403,2024-04-13,Burnley,Brighton & Hove Albion,1132,1237,Vincent Kompany,Roberto De Zerbi,S Hooper,1,...,0,79.7,0,1.5,1.5,6,61.9,15,5,1
4430,4095406,2024-04-13,Luton Town,Manchester City,1031,281,Rob Edwards,Pep Guardiola,J Brooks,1,...,0,76.0,0,0.3,0.1,7,44.4,18,8,0


In [64]:
club_value_by_game

,game_id,player_club_id,market_value_in_eur
0,3050167,11,3.064286e+07
1,3050167,281,6.357143e+07
2,3050168,603,2.196429e+06
3,3050168,989,6.035714e+06
4,3050169,873,1.117857e+07
...,...,...,...
4441,4095407,762,2.476000e+07
4442,4095408,543,1.526667e+07
4443,4095408,703,1.553333e+07
4444,4095409,379,2.371333e+07


In [65]:
master_df = pd.merge(master_df, club_value_by_game, left_on=['game_id', 'club_id'], right_on=['game_id', 'player_club_id'])

In [66]:
master_df

,game_id,Date,Team,Opponent,club_id,opponent_id,own_manager_name,opponent_manager_name,Referee,TY,...,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error,player_club_id,market_value_in_eur
0,3050167,2018-08-12,Arsenal,Manchester City,11,281,Unai Emery,Pep Guardiola,M Oliver,2,...,0,0.5,0.6,14,40.0,13,12,1,11,3.064286e+07
1,3050167,2018-08-12,Manchester City,Arsenal,281,11,Pep Guardiola,Unai Emery,M Oliver,2,...,2,0.5,0.4,5,28.6,14,12,1,281,6.357143e+07
2,3050168,2018-08-11,Cardiff City,Bournemouth,603,989,Neil Warnock,Eddie Howe,K Friend,1,...,0,1.5,1.2,12,47.6,16,15,0,603,2.196429e+06
3,3050168,2018-08-11,Bournemouth,Cardiff City,989,603,Eddie Howe,Neil Warnock,K Friend,1,...,2,1.3,1.1,5,47.1,11,4,0,989,6.035714e+06
4,3050169,2018-08-11,Fulham,Crystal Palace,931,873,Slavisa Jokanovic,Roy Hodgson,M Dean,1,...,0,0.5,0.7,19,42.9,5,15,1,931,1.060714e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4427,4095402,2024-04-13,Brentford,Sheffield Utd,1148,350,Thomas Frank,Chris Wilder,S Barrott,3,...,1,1.0,1.1,8,21.7,15,14,0,1148,1.618750e+07
4428,4095403,2024-04-13,Brighton & Hove Albion,Burnley,1237,1132,Roberto De Zerbi,Vincent Kompany,S Hooper,1,...,0,0.5,0.9,8,64.7,11,5,1,1237,1.813333e+07
4429,4095403,2024-04-13,Burnley,Brighton & Hove Albion,1132,1237,Vincent Kompany,Roberto De Zerbi,S Hooper,1,...,0,1.5,1.5,6,61.9,15,5,1,1132,9.333333e+06
4430,4095406,2024-04-13,Luton Town,Manchester City,1031,281,Rob Edwards,Pep Guardiola,J Brooks,1,...,0,0.3,0.1,7,44.4,18,8,0,1031,3.484615e+06


In [67]:
master_df.columns

Index(['game_id', 'Date', 'Team', 'Opponent', 'club_id', 'opponent_id',
       'own_manager_name', 'opponent_manager_name', 'Referee', 'TY', 'OY',
       'TR', 'OR', 'B365TW', 'B365D', 'B365OW', 'Table', 'Venue', 'Result',
       'Possesion', 'Aerial Duels(%)', 'GF', 'GA', 'Oppo. Table',
       'Shot on Target', 'Shot on Target(%)', 'Goals per Shot',
       'Expected Goals', 'Save%', 'Clean Sheet', 'Pass Completion %',
       'Assists', 'Exp. Assisted Goals', 'Expected Assists', 'Tackles Won',
       '% of Dribblers Tackled', 'Blocks', 'Interceptions', 'Error',
       'player_club_id', 'market_value_in_eur'],
      dtype='object')

In [68]:
master_df = master_df.drop('player_club_id', axis=1)

In [69]:
master_df.columns

Index(['game_id', 'Date', 'Team', 'Opponent', 'club_id', 'opponent_id',
       'own_manager_name', 'opponent_manager_name', 'Referee', 'TY', 'OY',
       'TR', 'OR', 'B365TW', 'B365D', 'B365OW', 'Table', 'Venue', 'Result',
       'Possesion', 'Aerial Duels(%)', 'GF', 'GA', 'Oppo. Table',
       'Shot on Target', 'Shot on Target(%)', 'Goals per Shot',
       'Expected Goals', 'Save%', 'Clean Sheet', 'Pass Completion %',
       'Assists', 'Exp. Assisted Goals', 'Expected Assists', 'Tackles Won',
       '% of Dribblers Tackled', 'Blocks', 'Interceptions', 'Error',
       'market_value_in_eur'],
      dtype='object')

In [70]:
master_df.isnull().sum()

game_id                   0
Date                      0
Team                      0
Opponent                  0
club_id                   0
opponent_id               0
own_manager_name          0
opponent_manager_name     0
Referee                   0
TY                        0
OY                        0
TR                        0
OR                        0
B365TW                    0
B365D                     0
B365OW                    0
Table                     0
Venue                     0
Result                    0
Possesion                 0
Aerial Duels(%)           0
GF                        0
GA                        0
Oppo. Table               0
Shot on Target            0
Shot on Target(%)         0
Goals per Shot            0
Expected Goals            0
Save%                     0
Clean Sheet               0
Pass Completion %         0
Assists                   0
Exp. Assisted Goals       0
Expected Assists          0
Tackles Won               0
% of Dribblers Tackl

In [71]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4432 entries, 0 to 4431
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   game_id                 4432 non-null   int64         
 1   Date                    4432 non-null   datetime64[ns]
 2   Team                    4432 non-null   object        
 3   Opponent                4432 non-null   object        
 4   club_id                 4432 non-null   int64         
 5   opponent_id             4432 non-null   int64         
 6   own_manager_name        4432 non-null   object        
 7   opponent_manager_name   4432 non-null   object        
 8   Referee                 4432 non-null   object        
 9   TY                      4432 non-null   int64         
 10  OY                      4432 non-null   int64         
 11  TR                      4432 non-null   int64         
 12  OR                      4432 non-null   int64   

In [72]:
master_df

,game_id,Date,Team,Opponent,club_id,opponent_id,own_manager_name,opponent_manager_name,Referee,TY,...,Pass Completion %,Assists,Exp. Assisted Goals,Expected Assists,Tackles Won,% of Dribblers Tackled,Blocks,Interceptions,Error,market_value_in_eur
0,3050167,2018-08-12,Arsenal,Manchester City,11,281,Unai Emery,Pep Guardiola,M Oliver,2,...,77.1,0,0.5,0.6,14,40.0,13,12,1,3.064286e+07
1,3050167,2018-08-12,Manchester City,Arsenal,281,11,Pep Guardiola,Unai Emery,M Oliver,2,...,81.9,2,0.5,0.4,5,28.6,14,12,1,6.357143e+07
2,3050168,2018-08-11,Cardiff City,Bournemouth,603,989,Neil Warnock,Eddie Howe,K Friend,1,...,58.4,0,1.5,1.2,12,47.6,16,15,0,2.196429e+06
3,3050168,2018-08-11,Bournemouth,Cardiff City,989,603,Eddie Howe,Neil Warnock,K Friend,1,...,79.0,2,1.3,1.1,5,47.1,11,4,0,6.035714e+06
4,3050169,2018-08-11,Fulham,Crystal Palace,931,873,Slavisa Jokanovic,Roy Hodgson,M Dean,1,...,85.2,0,0.5,0.7,19,42.9,5,15,1,1.060714e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4427,4095402,2024-04-13,Brentford,Sheffield Utd,1148,350,Thomas Frank,Chris Wilder,S Barrott,3,...,74.4,1,1.0,1.1,8,21.7,15,14,0,1.618750e+07
4428,4095403,2024-04-13,Brighton & Hove Albion,Burnley,1237,1132,Roberto De Zerbi,Vincent Kompany,S Hooper,1,...,84.2,0,0.5,0.9,8,64.7,11,5,1,1.813333e+07
4429,4095403,2024-04-13,Burnley,Brighton & Hove Albion,1132,1237,Vincent Kompany,Roberto De Zerbi,S Hooper,1,...,79.7,0,1.5,1.5,6,61.9,15,5,1,9.333333e+06
4430,4095406,2024-04-13,Luton Town,Manchester City,1031,281,Rob Edwards,Pep Guardiola,J Brooks,1,...,76.0,0,0.3,0.1,7,44.4,18,8,0,3.484615e+06


In [75]:
master_df = master_df.sort_values(['club_id', 'Date'])

In [76]:
len(master_df)

4432

In [76]:
# 여기까지 몸값만 추가된 것

In [77]:
# master_df.to_csv('non_rolling_mean.csv', index=False)

In [78]:
# 승률 컬럼 추가

In [79]:
#club_games_df = pd.read_csv('club_games.csv')

In [80]:
#club_games_df = club_games_df.sort_values(by=['club_id', 'game_id'])

In [81]:
# # 각 클럽의 총 게임 수 카운트 및 누적치로 추가
# club_games_df['total_games'] = club_games_df.groupby('club_id')['game_id'].cumcount()

# # 각 클럽의 승리한 게임 수 카운트 및 누적치로 추가
# club_games_df['wins'] = club_games_df.groupby('club_id')['is_win'].cumsum()
# club_games_df['wins'] = club_games_df.groupby('club_id')['wins'].shift(1)
# club_games_df['wins'] = club_games_df['wins'].fillna(0)

# # 승리 확률 계산 및 누적치로 추가
# club_games_df['win_percentage'] = (club_games_df['wins'] / club_games_df['total_games']) * 100
# club_games_df['win_percentage'] = club_games_df['win_percentage'].fillna(0)

In [82]:
# # 상대전적별 총 게임 수 카운트 및 누적치로 추가
# club_games_df['total_games_vs_opponent'] = club_games_df.groupby(['club_id', 'opponent_id'])['game_id'].cumcount()

# # 상대전적별 총 게임 수 카운트 및 누적치로 추가
# club_games_df['wins_vs_opponent'] = club_games_df.groupby(['club_id', 'opponent_id'])['is_win'].cumsum()
# club_games_df['wins_vs_opponent'] = club_games_df.groupby(['club_id', 'opponent_id'])['wins_vs_opponent'].shift(1)
# club_games_df['wins_vs_opponent'] = club_games_df['wins_vs_opponent'].fillna(0)

# # 승리 확률 계산 및 누적치로 추가
# club_games_df['win_percentage_vs_opponent'] = (club_games_df['wins_vs_opponent'] / club_games_df['total_games_vs_opponent']) * 100
# club_games_df['win_percentage_vs_opponent'] = club_games_df['win_percentage_vs_opponent'].fillna(0)

In [80]:
merged_df1 = merged_df1.sort_values(by=['club_id', 'date'])

In [82]:
# 각 클럽의 총 게임 수 카운트 및 누적치로 추가
merged_df1['total_games'] = merged_df1.groupby('club_id')['date'].cumcount()

# 각 클럽의 승리한 게임 수 카운트 및 누적치로 추가
merged_df1['wins'] = merged_df1.groupby('club_id')['is_win'].cumsum()
merged_df1['wins'] = merged_df1.groupby('club_id')['wins'].shift(1)
merged_df1['wins'] = merged_df1['wins'].fillna(0)

# 승리 확률 계산 및 누적치로 추가
merged_df1['win_percentage'] = (merged_df1['wins'] / merged_df1['total_games']) * 100
merged_df1['win_percentage'] = merged_df1['win_percentage'].fillna(0)

In [83]:
# 상대전적별 총 게임 수 카운트 및 누적치로 추가
merged_df1['total_games_vs_opponent'] = merged_df1.groupby(['club_id', 'opponent_id'])['date'].cumcount()

# 상대전적별 총 게임 수 카운트 및 누적치로 추가
merged_df1['wins_vs_opponent'] = merged_df1.groupby(['club_id', 'opponent_id'])['is_win'].cumsum()
merged_df1['wins_vs_opponent'] = merged_df1.groupby(['club_id', 'opponent_id'])['wins_vs_opponent'].shift(1)
merged_df1['wins_vs_opponent'] = merged_df1['wins_vs_opponent'].fillna(0)

# 승리 확률 계산 및 누적치로 추가
merged_df1['win_percentage_vs_opponent'] = (merged_df1['wins_vs_opponent'] / merged_df1['total_games_vs_opponent']) * 100
merged_df1['win_percentage_vs_opponent'] = merged_df1['win_percentage_vs_opponent'].fillna(0)

In [84]:
merged_df1

,game_id,date,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win,total_games,wins,win_percentage,total_games_vs_opponent,wins_vs_opponent,win_percentage_vs_opponent
933,3050167,2018-08-12,11,0,15.0,Unai Emery,281,2,3.0,Pep Guardiola,Home,0,0,0.0,0.000000,0,0.0,0.000000
2315,3050180,2018-08-18,11,2,17.0,Unai Emery,631,3,3.0,Maurizio Sarri,Away,0,1,0.0,0.000000,0,0.0,0.000000
1777,3050187,2018-08-25,11,3,9.0,Unai Emery,379,1,20.0,Manuel Pellegrini,Home,1,2,0.0,0.000000,0,0.0,0.000000
2225,3050199,2018-09-02,11,3,9.0,Unai Emery,603,2,16.0,Neil Warnock,Away,1,3,1.0,33.333333,0,0.0,0.000000
3159,3050212,2018-09-15,11,2,7.0,Unai Emery,762,1,19.0,Rafael Benítez,Away,1,4,2.0,50.000000,0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,4095353,2024-03-10,1237,1,8.0,Roberto De Zerbi,703,0,18.0,Nuno Espírito Santo,Home,1,217,67.0,30.875576,3,1.0,33.333333
4256,4095374,2024-03-31,1237,1,9.0,Roberto De Zerbi,31,2,1.0,Jürgen Klopp,Away,0,218,68.0,31.192661,11,2.0,18.181818
3154,4095382,2024-04-03,1237,0,9.0,Roberto De Zerbi,1148,0,15.0,Thomas Frank,Away,0,219,68.0,31.050228,5,3.0,60.000000
402,4095391,2024-04-06,1237,0,10.0,Roberto De Zerbi,11,3,1.0,Mikel Arteta,Home,0,220,68.0,30.909091,11,4.0,36.363636


In [85]:
master_df2 = pd.merge(master_df, merged_df1[['game_id', 'club_id', 'opponent_id', 'total_games', 'wins', 'win_percentage', 'total_games_vs_opponent', 'wins_vs_opponent', 'win_percentage_vs_opponent']], on=(['game_id', 'club_id', 'opponent_id']))

In [86]:
master_df2 = master_df2.sort_values(['club_id', 'Date'])

In [87]:
master_df2.isnull().sum()

game_id                       0
Date                          0
Team                          0
Opponent                      0
club_id                       0
opponent_id                   0
own_manager_name              0
opponent_manager_name         0
Referee                       0
TY                            0
OY                            0
TR                            0
OR                            0
B365TW                        0
B365D                         0
B365OW                        0
Table                         0
Venue                         0
Result                        0
Possesion                     0
Aerial Duels(%)               0
GF                            0
GA                            0
Oppo. Table                   0
Shot on Target                0
Shot on Target(%)             0
Goals per Shot                0
Expected Goals                0
Save%                         0
Clean Sheet                   0
Pass Completion %             0
Assists 

In [88]:
master_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4432 entries, 0 to 4431
Data columns (total 46 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   game_id                     4432 non-null   int64         
 1   Date                        4432 non-null   datetime64[ns]
 2   Team                        4432 non-null   object        
 3   Opponent                    4432 non-null   object        
 4   club_id                     4432 non-null   int64         
 5   opponent_id                 4432 non-null   int64         
 6   own_manager_name            4432 non-null   object        
 7   opponent_manager_name       4432 non-null   object        
 8   Referee                     4432 non-null   object        
 9   TY                          4432 non-null   int64         
 10  OY                          4432 non-null   int64         
 11  TR                          4432 non-null   int64       

In [89]:
master_df2

,game_id,Date,Team,Opponent,club_id,opponent_id,own_manager_name,opponent_manager_name,Referee,TY,...,Blocks,Interceptions,Error,market_value_in_eur,total_games,wins,win_percentage,total_games_vs_opponent,wins_vs_opponent,win_percentage_vs_opponent
0,3050167,2018-08-12,Arsenal,Manchester City,11,281,Unai Emery,Pep Guardiola,M Oliver,2,...,13,12,1,3.064286e+07,0,0.0,0.000000,0,0.0,0.000000
1,3050180,2018-08-18,Arsenal,Chelsea,11,631,Unai Emery,Maurizio Sarri,M Atkinson,2,...,12,12,2,3.235714e+07,1,0.0,0.000000,0,0.0,0.000000
2,3050187,2018-08-25,Arsenal,West Ham,11,379,Unai Emery,Manuel Pellegrini,G Scott,1,...,14,6,1,3.021429e+07,2,0.0,0.000000,0,0.0,0.000000
3,3050199,2018-09-02,Arsenal,Cardiff City,11,603,Unai Emery,Neil Warnock,A Taylor,4,...,6,11,2,3.235714e+07,3,1.0,33.333333,0,0.0,0.000000
4,3050212,2018-09-15,Arsenal,Newcastle Utd,11,762,Unai Emery,Rafael Benítez,L Probert,0,...,7,7,0,3.235714e+07,4,2.0,50.000000,0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4427,4095353,2024-03-10,Brighton & Hove Albion,Nottingham Forest,1237,703,Roberto De Zerbi,Nuno Espírito Santo,M Salisbury,4,...,9,5,1,1.906250e+07,217,67.0,30.875576,3,1.0,33.333333
4428,4095374,2024-03-31,Brighton & Hove Albion,Liverpool,1237,31,Roberto De Zerbi,Jürgen Klopp,D Coote,3,...,18,9,0,1.766667e+07,218,68.0,31.192661,11,2.0,18.181818
4429,4095382,2024-04-03,Brighton & Hove Albion,Brentford,1237,1148,Roberto De Zerbi,Thomas Frank,A Madley,1,...,9,8,0,1.606667e+07,219,68.0,31.050228,5,3.0,60.000000
4430,4095391,2024-04-06,Brighton & Hove Albion,Arsenal,1237,11,Roberto De Zerbi,Mikel Arteta,J Brooks,1,...,17,10,0,1.985714e+07,220,68.0,30.909091,11,4.0,36.363636


In [90]:
master_df2.to_csv('master_df(train+validation).csv', index = False)